## Lexicase Selection注意事项

对于Lexicase Selection，适应度评估需要更改为返回多个误差组成的向量，而不是均方误差（MSE）。这样，Lexicase Selection才能独立考虑每个个体在每个测试样本上的表现，从而提高选择的多样性。

In [1]:
import numpy as np
import math
import operator

from deap import base, creator, tools, gp


# 符号回归
def evalSymbReg(individual, pset):
    # 编译GP树为函数
    func = gp.compile(expr=individual, pset=pset)
    
    # 使用numpy创建一个向量
    x = np.linspace(-10, 10, 100) 
    
    return tuple((func(x) - x**2)**2)


# 创建个体和适应度函数，适应度数组大小与数据量相同
creator.create("FitnessMin", base.Fitness, weights=(-1.0,) * 100)  # 假设我们有20个数据点
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

### 遗传算子
选择算子需要改成Lexicase Selection，其他不需要改变。对于回归问题，需要使用AutomaticEpsilonLexicase。而对于分类问题，则使用Lexicase即可。

In [2]:
import random

# 定义函数集合和终端集合
pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", lambda: random.randint(-1, 1))
pset.renameArguments(ARG0='x')

# 定义遗传编程操作
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evalSymbReg, pset=pset)
toolbox.register("select", tools.selAutomaticEpsilonLexicase)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

C:\Users\zhenl\anaconda3\Lib\site-packages\deap\gp.py:254: RuntimeWarning: Ephemeral rand101 function cannot be pickled because its generating function is a lambda function. Use functools.partial instead.
  warnings.warn("Ephemeral {name} function cannot be "


In [3]:
import numpy
from deap import algorithms

# 定义统计指标
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register("avg", numpy.mean)
mstats.register("std", numpy.std)
mstats.register("min", numpy.min)
mstats.register("max", numpy.max)

# 使用默认算法
population = toolbox.population(n=300)
hof = tools.HallOfFame(1)
pop, log  = algorithms.eaSimple(population=population,
                           toolbox=toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=mstats, halloffame=hof, verbose=True)
print(str(hof[0]))


   	      	                        fitness                        	                     size                     
   	      	-------------------------------------------------------	----------------------------------------------
gen	nevals	avg    	gen	max     	min	nevals	std    	avg    	gen	max	min	nevals	std   
0  	300   	3331.11	0  	1.44e+06	0  	300   	25891.8	4.00333	0  	7  	2  	300   	1.6763
1  	178   	8053   	1  	4.41e+06	0  	178   	88855.6	4.3    	1  	11 	2  	178   	1.76163
2  	179   	43482.7	2  	1.0201e+08	0  	179   	1.54764e+06	4.30333	2  	13 	2  	179   	1.88979
3  	179   	1999.24	3  	1.21e+06  	0  	179   	24746.1    	4.1    	3  	17 	2  	179   	1.7767 
4  	187   	1545.33	4  	1.21e+06  	0  	187   	18116.9    	4.09333	4  	13 	2  	187   	1.72181
5  	175   	43358  	5  	9.801e+07 	0  	175   	1.48069e+06	4.13   	5  	15 	2  	175   	1.96632
6  	200   	83445.2	6  	1.0201e+08	0  	200   	2.14078e+06	4.30667	6  	13 	2  	200   	2.24038
7  	191   	2128.79	7  	1.21e+06  	0  	191   	30490.1    

KeyboardInterrupt: 